<a href="https://colab.research.google.com/github/WilliamAshbee/splineexample/blob/main/SeqMLP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

With progressive growing of the sequence, the loss drops more stably. 

In [1]:
#TODO:TEST A DIFFERENT TYPE OF WRITE HEAD (ADD A SECOND LTM MEMORY MODULE)
#TODO:TEST A DIFFERENT TYPE OF LOSS TO SMOOTH OUT THE BLURRY READ/WRITE.

https://r2rt.com/non-zero-initial-states-for-recurrent-neural-networks.html

In [2]:
import torch
import numpy as np
import pylab as plt
from skimage import filters
import math

global numpoints
numpoints = 1000
side = 32

rows = torch.zeros(32,32)
columns = torch.zeros(32,32)


for i in range(32):
    columns[:,i] = i
    rows[i,:] = i


def donut_matrix(length = 10):

    radiusMax = side /3
    w = 1
    sigmas = [None, 1]
    
    canvas = torch.zeros((length,side, side))
    r0 = torch.tensor(np.random.uniform(side/4, side/3, length))

    radii = torch.zeros((length,numpoints))
    radii[:, :] = r0.unsqueeze(1)
    
    ind = [x for x in range(numpoints)]

    theta = torch.FloatTensor(ind)
    theta *= math.pi*2.0/(float)(numpoints)
    
    for i in range(1,length):
        a = np.random.uniform(1.0,3.0)*torch.sin(np.random.uniform(20.0)*theta+np.random.uniform(1000.0))
        #a = 4.0*torch.sin(10.0*theta)
        #print(a.shape,torch.max(a))
        radii[i,:] += a
        #print(radii.shape, torch.max(radii))
    
    assert torch.min(radii)>0
    #print(radii.max(axis = 0)[0].shape)
    rmaxs = radii.max(axis = 1)[0]
    pmins = rmaxs+1.0
    pmaxs = side-rmaxs-1.0
    x0 = np.random.uniform(pmins,pmaxs)
    y0 = np.random.uniform(pmins,pmaxs)
    x0[:]=side/2
    y0[:]=side/2
    x0 = torch.tensor(x0)
    y0 = torch.tensor(y0)
    
    x0 = x0.unsqueeze(1)
    y0 = y0.unsqueeze(1)
    #radii = torch.from_numpy(radii)
    xrfactors = torch.cos(theta).unsqueeze(0)
    yrfactors = torch.sin(theta).unsqueeze(0)
    
    print('x0_y0_r_xrf_yrf',x0.shape,y0.shape,radii.shape,xrfactors.shape,yrfactors.shape)

    x = (x0+(xrfactors*radii))
    y = (y0+(yrfactors*radii))
    assert x.shape == (length,numpoints)
    assert y.shape == (length,numpoints)
    assert torch.sum(x[x>(side-1)])==0 
    assert torch.sum(x[x<0])==0 
    assert torch.sum(y[y>(side-1)])==0 
    assert torch.sum(y[y<0])==0 
    
    points = torch.zeros(length,numpoints,2)
    for l in range(length):
        canvas[l,y[l,:].type(torch.LongTensor),x[l,:].type(torch.LongTensor)]=1.0
        points[l,:,0] = x[l,:]#modified for lstm discriminator
        points[l,:,1] = y[l,:]#modified for lstm discriminator 
    
    
    return {
        'canvas': canvas, 
        'points':points.type(torch.FloatTensor)}

def plot_all_model( sample = None, label=None,fig=None,ax=None):
    X = label[:,0]
    Y = label[:,1]
    img = sample[:,:].detach().squeeze().cpu().numpy()
    ax.imshow(img, cmap=plt.cm.gray_r)
    predres = 1000

    s = [.001 for x in range(predres)]
    
    assert len(s) == predres
    c = ['red' for x in range(predres)]
    assert len(c) == predres
    ax.scatter(X.detach().cpu().numpy(),Y.detach().cpu().numpy(),s = s,c = c)


def getXYs(model=None,loader_disp = None):

    outputs = torch.zeros(100,1000,2).cuda()
    h0 = None
    c0 = None
    
    for samples,labels in loader_disp:
      for i in range(38):
        optimizer.zero_grad()
        if i == 0:
          out, h0, c0,o,output = model(samples.cuda(),labels[:,0,:2].cuda(),h0,c0,ind = i)
          outputs[:,i*25:(i+1)*25,:] = out.detach()
        else:
          out, h0, c0,o,output = model(samples.cuda(),labels[:,0,:2].cuda(),h0,c0,ind = i,o = o,output=output)
          outputs[:,i*25:(i+1)*25,:] = out.detach()

      return samples, outputs


          
        

def plot_all( sample = None, labels = None,fig = None,ax = None):
    img = sample[:,:].squeeze().cpu().numpy()
    ax.imshow(img, cmap=plt.cm.gray_r)
    X = labels[:,0]
    Y = labels[:,1]
    s = [.001 for x in range(numpoints)]
    c = ['red' for x in range(numpoints)]
    ax.scatter(X.cpu().numpy(),Y.cpu().numpy(),s = s,c = c)
    
class DonutDataset(torch.utils.data.Dataset):
    """Donut dataset."""
    def __init__(self, length = 10):
        """
        Args:
            csv_file (string): Path to the csv file with annotations.
            root_dir (string): Directory with all the images.
            transform (callable, optional): Optional transform to be applied
                on a sample.
        """
        self.length = length
        self.values = donut_matrix(length)
        assert self.values['canvas'].shape[0] == self.length
        assert self.values['points'].shape[0] == self.length
        
        count = 0
        for i in range(self.length):
          a = self[i]
          c = a[0]
          for el in a[1]:
            #print(c[(int)(el[1]),(int)(el[0])].item())
            #assert c[(int)(el[1]),(int)(el[0])].item() == 1
            y,x = (int)(el[1]),(int)(el[0])
            if x < side-2 and x > 2 and y < side-2 and y > 2: 
              if c[y,x] != 1 and \
                c[y+1,x] != 1 and c[y+1,-1+x] != 1 and c[y+1,1+x] != 1 and \
                c[y-1,x] != 1 and c[y,-1+x] != 1 and c[y,1+x] != 1:
                count+=1
        assert count ==0
    def __len__(self):
        return self.length


    def __getitem__(self, idx):
        canvas = self.values["canvas"]
        
        canvas = canvas[idx,:,:]
        assert canvas.shape == (side,side)
        
        points = self.values["points"]
        points = points[idx,:]
        #points = points.unsqueeze(1)
        #z = torch.zeros(numpoints,1)
        #print(z.shape)
        #points = torch.cat([points,z], dim = 1)
        
        #print('points', points.shape)
        return canvas, points
    
    @staticmethod
    def displayCanvas(title,ddataset, model):
        #model.setBatchSize(batch_size = 1)
        fig = plt.figure()
          
        if model != None:
          #model = model.eval()
          loader_disp = data.DataLoader(
            ddataset, 
            batch_size=100,
            num_workers=2)
          samples, outputs = getXYs(model=model,loader_disp = loader_disp)
          for i in range(100):
            ax = fig.add_subplot(10,10,i+1)
            plot_all_model(samples[i,:,:],outputs[i,:,:],fig=fig,ax=ax)
            plt.axis('off')

        else:
          for i in range(100):
            sample, labels = dataset[i]
            ax = fig.add_subplot(10,10,i+1)
            plot_all(sample = sample,labels = labels,fig = fig, ax= ax)
            plt.axis('off')

        fig.savefig(title,dpi=450)
        plt.close(fig) #where f is the figure
        plt.clf()
        plt.cla()
        plt.close()#should free memory
        print('open file: ',title)


In [3]:
dataset = DonutDataset(length = 100)

DonutDataset.displayCanvas('donut.png',dataset, model = None)


x0_y0_r_xrf_yrf torch.Size([100, 1]) torch.Size([100, 1]) torch.Size([100, 1000]) torch.Size([1, 1000]) torch.Size([1, 1000])
open file:  donut.png


In [4]:

from torch.utils import data
from torch.utils.data import DataLoader, TensorDataset, RandomSampler

mini_batch = 100
dataset = DonutDataset(length = 100*20)
loader_train = data.DataLoader(
    dataset, 
    batch_size=mini_batch,
    sampler=RandomSampler(data_source=dataset),
    num_workers=2)

x0_y0_r_xrf_yrf torch.Size([2000, 1]) torch.Size([2000, 1]) torch.Size([2000, 1000]) torch.Size([1, 1000]) torch.Size([1, 1000])


In [5]:
mini_batch = 100
test_dataset = DonutDataset(length = 1000)
loader_test = data.DataLoader(
    test_dataset, 
    batch_size=mini_batch,
    sampler=RandomSampler(data_source=test_dataset),
    num_workers=2)

x0_y0_r_xrf_yrf torch.Size([1000, 1]) torch.Size([1000, 1]) torch.Size([1000, 1000]) torch.Size([1, 1000]) torch.Size([1, 1000])


In [6]:
pred_dataset = DonutDataset(length = 100)

x0_y0_r_xrf_yrf torch.Size([100, 1]) torch.Size([100, 1]) torch.Size([100, 1000]) torch.Size([1, 1000]) torch.Size([1, 1000])


In [49]:
import torch
import torch.nn as nn
from torch.hub import load_state_dict_from_url

class DecoderRNN(nn.Module):
    def __init__(self):
        super(DecoderRNN, self).__init__()
        self.track = 0
        # define the properties
        self.embed_size = 2
        self.hidden_size = 256
        self.num_layers = 2
        self.seq_len = 25
        self.full_len = 1000
        self.longmem_len = int(self.full_len/self.seq_len)
        self.longtermMemP = None


        
        self.fc_out = nn.Sequential(
            nn.Linear(2+2*1000+1024,self.hidden_size*2),
            nn.ReLU(),
            nn.Linear(self.hidden_size*2,10000),
            nn.ReLU(),
            nn.Dropout(),
            nn.Linear(10000,2),
            nn.Sigmoid()
        )

        
        
    def forward(self, features, p0,h0 = None, c0= None, ind = 0, o = None,output = None):
        
        assert features != None
        self.track +=1
        batch_size = features.size(0)
        

        
        
        if o == None:
          out = p0
          self.longtermMemP = torch.zeros(batch_size,1000,2).cuda()
        else:
          #print('hi')
          out = o

        outputs = torch.empty((batch_size, self.seq_len, 2)).cuda()
        for t in range(self.seq_len):
        
            
            self.longtermMemP[:,self.seq_len*ind+t,:] = out.detach()

            combOut = torch.cat([out,torch.flatten(features,start_dim=1),torch.flatten(self.longtermMemP,start_dim=1)],dim=1)
            
            out = self.fc_out(combOut)
            out = out*32.0
            outputs[:, t, :] = out
            
        
            
        #self.embs = self.embs.detach().clone()
        return outputs, None, None, out.detach(),None#, captions


In [50]:
model = DecoderRNN().cuda()

In [51]:
optimizer = torch.optim.Adam(model.parameters(),lr = 0.0001, betas = (.9,.999))#ideal


In [52]:
import torch
from torch.utils.tensorboard import SummaryWriter
writer = SummaryWriter()


In [53]:
for seql in range(3,23,4):
  #seql = 19
  for epoch in range (2):
    loss = None
    #model = model.train()

    for x,y in loader_train:
      h0 = None
      c0 = None
      for i in range(seql):
        x = x.cuda()
        y = y.cuda()
        
        optimizer.zero_grad()
        
        if i == 0:
          out, h0, c0,o,output = model(x,y[:,0,:2],h0,c0,ind = i)
        else:
          out, h0, c0,o,output = model(x,y[:,0,:2],h0,c0,ind = i,o = o,output=output)
        loss = torch.mean((out-y[:,1+(25*i):1+(25*(i+1)),:2])**2)
        #writer.add_scalar("Loss/train", loss, epoch)
        
        loss.backward()
        optimizer.step()
      #print(y[:,0,:2])
      print('epoch',epoch,'seql',seql)
      print('train loss', loss)
    
    with torch.no_grad():
      #model = model.eval()
      for x,y in loader_test:
        h0 = None
        c0 = None
        for i in range(seql):
          x = x.cuda()
          y = y.cuda()
          
          if i == 0:
            out, h0, c0,o,output = model(x,y[:,0,:2],h0,c0,ind = i)
          else:
            out, h0, c0,o,output = model(x,y[:,0,:2],h0,c0,ind = i,o = o,output=output)
          
          loss = torch.mean((out-y[:,1+(25*i):1+(25*(i+1)),:2])**2)
        
          assert not torch.any(torch.isnan(out)).item()
          assert not torch.any(torch.isnan(y)).item()
        
        print('epoch',epoch,'seql',seql)
        print('testloss',loss)
  
        

epoch 0 seql 3
train loss tensor(24.6565, device='cuda:0', grad_fn=<MeanBackward0>)
epoch 0 seql 3
train loss tensor(61.5909, device='cuda:0', grad_fn=<MeanBackward0>)
epoch 0 seql 3
train loss tensor(39.9300, device='cuda:0', grad_fn=<MeanBackward0>)
epoch 0 seql 3
train loss tensor(15.2554, device='cuda:0', grad_fn=<MeanBackward0>)
epoch 0 seql 3
train loss tensor(17.3874, device='cuda:0', grad_fn=<MeanBackward0>)
epoch 0 seql 3
train loss tensor(13.1934, device='cuda:0', grad_fn=<MeanBackward0>)
epoch 0 seql 3
train loss tensor(5.3471, device='cuda:0', grad_fn=<MeanBackward0>)
epoch 0 seql 3
train loss tensor(6.0154, device='cuda:0', grad_fn=<MeanBackward0>)
epoch 0 seql 3
train loss tensor(5.4031, device='cuda:0', grad_fn=<MeanBackward0>)
epoch 0 seql 3
train loss tensor(5.3338, device='cuda:0', grad_fn=<MeanBackward0>)
epoch 0 seql 3
train loss tensor(3.8116, device='cuda:0', grad_fn=<MeanBackward0>)
epoch 0 seql 3
train loss tensor(4.4905, device='cuda:0', grad_fn=<MeanBackward0>

In [54]:
with torch.no_grad():
  #model = model.eval()
  DonutDataset.displayCanvas('intermediate1.png',pred_dataset, model = model)

open file:  intermediate1.png


In [55]:
#optimizer = torch.optim.Adam(model.parameters(),lr = 0.0001, betas = (.9,.999))#ideal

for g in optimizer.param_groups:
    g['lr'] = 0.0001



In [56]:
#for seql in range(15,20,4):
seql = 23
for epoch in range (10):
  #model = model.train()
  loss = None
  for x,y in loader_train:
    h0 = None
    c0 = None
    for i in range(seql):
      x = x.cuda()
      y = y.cuda()
      
      optimizer.zero_grad()
      
      if i == 0:
        out, h0, c0,o,output = model(x,y[:,0,:2],h0,c0,ind = i)
      else:
        out, h0, c0,o,output = model(x,y[:,0,:2],h0,c0,ind = i,o = o,output=output)
      loss = torch.mean((out-y[:,1+(25*i):1+(25*(i+1)),:2])**2)
      writer.add_scalar("Loss/train", loss, epoch)
      
      loss.backward()
      optimizer.step()
    #print(y[:,0,:2])
    print('epoch',epoch,'seql',seql)
    print(loss)
  with torch.no_grad():
    #model = model.eval()
    for x,y in loader_test:
      h0 = None
      c0 = None
      for i in range(seql):
        x = x.cuda()
        y = y.cuda()
        
        if i == 0:
          out, h0, c0,o,output = model(x,y[:,0,:2],h0,c0,ind = i)
        else:
          out, h0, c0,o,output = model(x,y[:,0,:2],h0,c0,ind = i,o = o,output=output)
        
        loss = torch.mean((out-y[:,1+(25*i):1+(25*(i+1)),:2])**2)
      
        assert not torch.any(torch.isnan(out)).item()
        assert not torch.any(torch.isnan(y)).item()
      
      print('epoch',epoch,'seql',seql)
      print('testloss',loss)


epoch 0 seql 23
tensor(34.9822, device='cuda:0', grad_fn=<MeanBackward0>)
epoch 0 seql 23
tensor(5.1072, device='cuda:0', grad_fn=<MeanBackward0>)
epoch 0 seql 23
tensor(18.6169, device='cuda:0', grad_fn=<MeanBackward0>)
epoch 0 seql 23
tensor(28.9360, device='cuda:0', grad_fn=<MeanBackward0>)
epoch 0 seql 23
tensor(4.0642, device='cuda:0', grad_fn=<MeanBackward0>)
epoch 0 seql 23
tensor(10.8055, device='cuda:0', grad_fn=<MeanBackward0>)
epoch 0 seql 23
tensor(4.5362, device='cuda:0', grad_fn=<MeanBackward0>)
epoch 0 seql 23
tensor(47.1531, device='cuda:0', grad_fn=<MeanBackward0>)
epoch 0 seql 23
tensor(40.3086, device='cuda:0', grad_fn=<MeanBackward0>)
epoch 0 seql 23
tensor(37.1390, device='cuda:0', grad_fn=<MeanBackward0>)
epoch 0 seql 23
tensor(30.8209, device='cuda:0', grad_fn=<MeanBackward0>)
epoch 0 seql 23
tensor(28.6202, device='cuda:0', grad_fn=<MeanBackward0>)
epoch 0 seql 23
tensor(25.4155, device='cuda:0', grad_fn=<MeanBackward0>)
epoch 0 seql 23
tensor(23.9358, device='c

In [57]:
torch.save(model.state_dict(), "lastmodel.pth")


In [58]:
with torch.no_grad():
  #model = model.eval()
  DonutDataset.displayCanvas('intermediate2.png',pred_dataset, model = model)

open file:  intermediate2.png


In [59]:
#optimizer = torch.optim.Adam(model.parameters(),lr = 0.00001, betas = (.9,.999))#ideal
for g in optimizer.param_groups:
    g['lr'] = 0.00001



In [60]:

#model = model.train()
#for seql in range(15,20,4):
seql = 23
for epoch in range (20):
  loss = None
  #model = model.train()
  for x,y in loader_train:
    h0 = None
    c0 = None
    for i in range(seql):
      x = x.cuda()
      y = y.cuda()
      
      optimizer.zero_grad()
      
      if i == 0:
        out, h0, c0,o,output = model(x,y[:,0,:2],h0,c0,ind = i)
      else:
        out, h0, c0,o,output = model(x,y[:,0,:2],h0,c0,ind = i,o = o,output=output)
      loss = torch.mean((out-y[:,1+(25*i):1+(25*(i+1)),:2])**2)
      writer.add_scalar("Loss/train", loss, epoch)
      
      loss.backward()
      optimizer.step()
    #print(y[:,0,:2])
    print('epoch',epoch,'seql',seql)
    print(loss)
  with torch.no_grad():
      #model = model.eval()
      for x,y in loader_test:
        h0 = None
        c0 = None
        for i in range(seql):
          x = x.cuda()
          y = y.cuda()
          
          if i == 0:
            out, h0, c0,o,output = model(x,y[:,0,:2],h0,c0,ind = i)
          else:
            out, h0, c0,o,output = model(x,y[:,0,:2],h0,c0,ind = i,o = o,output=output)
          
          loss = torch.mean((out-y[:,1+(25*i):1+(25*(i+1)),:2])**2)
        
          assert not torch.any(torch.isnan(out)).item()
          assert not torch.any(torch.isnan(y)).item()
        
        print('epoch',epoch,'seql',seql)
        print('testloss',loss)
  
        

epoch 0 seql 23
tensor(2.4721, device='cuda:0', grad_fn=<MeanBackward0>)
epoch 0 seql 23
tensor(2.4499, device='cuda:0', grad_fn=<MeanBackward0>)
epoch 0 seql 23
tensor(2.4818, device='cuda:0', grad_fn=<MeanBackward0>)
epoch 0 seql 23
tensor(2.3237, device='cuda:0', grad_fn=<MeanBackward0>)
epoch 0 seql 23
tensor(2.2200, device='cuda:0', grad_fn=<MeanBackward0>)
epoch 0 seql 23
tensor(2.2643, device='cuda:0', grad_fn=<MeanBackward0>)
epoch 0 seql 23
tensor(2.3799, device='cuda:0', grad_fn=<MeanBackward0>)
epoch 0 seql 23
tensor(2.1101, device='cuda:0', grad_fn=<MeanBackward0>)
epoch 0 seql 23
tensor(2.2250, device='cuda:0', grad_fn=<MeanBackward0>)
epoch 0 seql 23
tensor(2.2459, device='cuda:0', grad_fn=<MeanBackward0>)
epoch 0 seql 23
tensor(2.4438, device='cuda:0', grad_fn=<MeanBackward0>)
epoch 0 seql 23
tensor(2.2502, device='cuda:0', grad_fn=<MeanBackward0>)
epoch 0 seql 23
tensor(2.4310, device='cuda:0', grad_fn=<MeanBackward0>)
epoch 0 seql 23
tensor(2.2540, device='cuda:0', gra

In [61]:
torch.save(model.state_dict(), "lastmodel.pth")


ran so well, interrupted

In [62]:
with torch.no_grad():
  #model = model.eval()
  DonutDataset.displayCanvas('intermediate3.png',pred_dataset, model = model)

open file:  intermediate3.png


In [63]:
#optimizer = torch.optim.Adam(model.parameters(),lr = 0.00001, betas = (.9,.999))#ideal
#for g in optimizer.param_groups:
#    g['lr'] = 0.0001



In [64]:
for seql in range(23,39,2):
  #seql = 19
  for epoch in range (6):
    loss = None
    #model = model.train()

    for x,y in loader_train:
      h0 = None
      c0 = None
      for i in range(seql):
        x = x.cuda()
        y = y.cuda()
        
        optimizer.zero_grad()
        
        if i == 0:
          out, h0, c0,o,output = model(x,y[:,0,:2],h0,c0,ind = i)
        else:
          out, h0, c0,o,output = model(x,y[:,0,:2],h0,c0,ind = i,o = o,output=output)
        loss = torch.mean((out-y[:,1+(25*i):1+(25*(i+1)),:2])**2)
        #writer.add_scalar("Loss/train", loss, epoch)
        
        loss.backward()
        optimizer.step()
      #print(y[:,0,:2])
      print('epoch',epoch,'seql',seql)
      print('train loss', loss)
    
    with torch.no_grad():
      #model = model.eval()
      for x,y in loader_test:
        h0 = None
        c0 = None
        for i in range(seql):
          x = x.cuda()
          y = y.cuda()
          
          if i == 0:
            out, h0, c0,o,output = model(x,y[:,0,:2],h0,c0,ind = i)
          else:
            out, h0, c0,o,output = model(x,y[:,0,:2],h0,c0,ind = i,o = o,output=output)
          
          loss = torch.mean((out-y[:,1+(25*i):1+(25*(i+1)),:2])**2)
        
          assert not torch.any(torch.isnan(out)).item()
          assert not torch.any(torch.isnan(y)).item()
        
        print('epoch',epoch,'seql',seql)
        print('testloss',loss)
  
        

epoch 0 seql 23
train loss tensor(1.4055, device='cuda:0', grad_fn=<MeanBackward0>)
epoch 0 seql 23
train loss tensor(1.6443, device='cuda:0', grad_fn=<MeanBackward0>)
epoch 0 seql 23
train loss tensor(1.6184, device='cuda:0', grad_fn=<MeanBackward0>)
epoch 0 seql 23
train loss tensor(1.6476, device='cuda:0', grad_fn=<MeanBackward0>)
epoch 0 seql 23
train loss tensor(1.5350, device='cuda:0', grad_fn=<MeanBackward0>)
epoch 0 seql 23
train loss tensor(1.6383, device='cuda:0', grad_fn=<MeanBackward0>)
epoch 0 seql 23
train loss tensor(1.5062, device='cuda:0', grad_fn=<MeanBackward0>)
epoch 0 seql 23
train loss tensor(1.5823, device='cuda:0', grad_fn=<MeanBackward0>)
epoch 0 seql 23
train loss tensor(1.4472, device='cuda:0', grad_fn=<MeanBackward0>)
epoch 0 seql 23
train loss tensor(1.6154, device='cuda:0', grad_fn=<MeanBackward0>)
epoch 0 seql 23
train loss tensor(1.5743, device='cuda:0', grad_fn=<MeanBackward0>)
epoch 0 seql 23
train loss tensor(1.3583, device='cuda:0', grad_fn=<MeanBack

In [65]:
torch.save(model.state_dict(), "lastmodel.pth")


In [66]:
with torch.no_grad():
  #model = model.eval()
  DonutDataset.displayCanvas('intermediate4.png',pred_dataset, model = model)

open file:  intermediate4.png


In [67]:
#optimizer = torch.optim.Adam(model.parameters(),lr = 0.00001, betas = (.9,.999))#ideal
#for g in optimizer.param_groups:
#    g['lr'] = 0.0001



In [68]:
#for seql in range(15,20,4):
seql = 38
for epoch in range (20):
  #model = model.train()
  loss = None
  for x,y in loader_train:
    h0 = None
    c0 = None
    for i in range(seql):
      x = x.cuda()
      y = y.cuda()
      
      optimizer.zero_grad()
      
      if i == 0:
        out, h0, c0,o,output = model(x,y[:,0,:2],h0,c0,ind = i)
      else:
        out, h0, c0,o,output = model(x,y[:,0,:2],h0,c0,ind = i,o = o,output=output)
      loss = torch.mean((out-y[:,1+(25*i):1+(25*(i+1)),:2])**2)
      writer.add_scalar("Loss/train", loss, epoch)
      
      loss.backward()
      optimizer.step()
    #print(y[:,0,:2])
    print('epoch',epoch,'seql',seql)
    print(loss)
  with torch.no_grad():
    #model = model.eval()
    for x,y in loader_test:
      h0 = None
      c0 = None
      for i in range(seql):
        x = x.cuda()
        y = y.cuda()
        
        if i == 0:
          out, h0, c0,o,output = model(x,y[:,0,:2],h0,c0,ind = i)
        else:
          out, h0, c0,o,output = model(x,y[:,0,:2],h0,c0,ind = i,o = o,output=output)
        
        loss = torch.mean((out-y[:,1+(25*i):1+(25*(i+1)),:2])**2)
      
        assert not torch.any(torch.isnan(out)).item()
        assert not torch.any(torch.isnan(y)).item()
      
      print('epoch',epoch,'seql',seql)
      print('testloss',loss)


epoch 0 seql 38
tensor(4.3341, device='cuda:0', grad_fn=<MeanBackward0>)
epoch 0 seql 38
tensor(6.0531, device='cuda:0', grad_fn=<MeanBackward0>)
epoch 0 seql 38
tensor(2.6688, device='cuda:0', grad_fn=<MeanBackward0>)
epoch 0 seql 38
tensor(5.6077, device='cuda:0', grad_fn=<MeanBackward0>)
epoch 0 seql 38
tensor(2.6771, device='cuda:0', grad_fn=<MeanBackward0>)
epoch 0 seql 38
tensor(9.5687, device='cuda:0', grad_fn=<MeanBackward0>)
epoch 0 seql 38
tensor(2.6209, device='cuda:0', grad_fn=<MeanBackward0>)
epoch 0 seql 38
tensor(3.6530, device='cuda:0', grad_fn=<MeanBackward0>)
epoch 0 seql 38
tensor(2.7153, device='cuda:0', grad_fn=<MeanBackward0>)
epoch 0 seql 38
tensor(6.1473, device='cuda:0', grad_fn=<MeanBackward0>)
epoch 0 seql 38
tensor(3.1520, device='cuda:0', grad_fn=<MeanBackward0>)
epoch 0 seql 38
tensor(10.0331, device='cuda:0', grad_fn=<MeanBackward0>)
epoch 0 seql 38
tensor(3.7724, device='cuda:0', grad_fn=<MeanBackward0>)
epoch 0 seql 38
tensor(3.4554, device='cuda:0', gr

In [69]:
torch.save(model.state_dict(), "lastmodel.pth")


In [70]:
with torch.no_grad():
  #model = model.eval()
  DonutDataset.displayCanvas('intermediate5.png',pred_dataset, model = model)

open file:  intermediate5.png


In [74]:
#optimizer = torch.optim.Adam(model.parameters(),lr = 0.00001, betas = (.9,.999))#ideal
for g in optimizer.param_groups:
    g['lr'] = 0.000005



In [75]:

#model = model.train()
#for seql in range(15,20,4):
seql = 38
for epoch in range (120):
  loss = None
  #model = model.train()
  for x,y in loader_train:
    h0 = None
    c0 = None
    for i in range(seql):
      x = x.cuda()
      y = y.cuda()
      
      optimizer.zero_grad()
      
      if i == 0:
        out, h0, c0,o,output = model(x,y[:,0,:2],h0,c0,ind = i)
      else:
        out, h0, c0,o,output = model(x,y[:,0,:2],h0,c0,ind = i,o = o,output=output)
      loss = torch.mean((out-y[:,1+(25*i):1+(25*(i+1)),:2])**2)
      writer.add_scalar("Loss/train", loss, epoch)
      
      loss.backward()
      optimizer.step()
    #print(y[:,0,:2])
    print('epoch',epoch,'seql',seql)
    print(loss)
  with torch.no_grad():
      #model = model.eval()
      for x,y in loader_test:
        h0 = None
        c0 = None
        for i in range(seql):
          x = x.cuda()
          y = y.cuda()
          
          if i == 0:
            out, h0, c0,o,output = model(x,y[:,0,:2],h0,c0,ind = i)
          else:
            out, h0, c0,o,output = model(x,y[:,0,:2],h0,c0,ind = i,o = o,output=output)
          
          loss = torch.mean((out-y[:,1+(25*i):1+(25*(i+1)),:2])**2)
        
          assert not torch.any(torch.isnan(out)).item()
          assert not torch.any(torch.isnan(y)).item()
        
        print('epoch',epoch,'seql',seql)
        print('testloss',loss)
  
        

Streaming output truncated to the last 5000 lines.
epoch 36 seql 38
testloss tensor(1.6580, device='cuda:0')
epoch 36 seql 38
testloss tensor(1.4954, device='cuda:0')
epoch 36 seql 38
testloss tensor(1.4130, device='cuda:0')
epoch 36 seql 38
testloss tensor(1.3245, device='cuda:0')
epoch 36 seql 38
testloss tensor(1.3121, device='cuda:0')
epoch 36 seql 38
testloss tensor(1.9277, device='cuda:0')
epoch 36 seql 38
testloss tensor(1.5938, device='cuda:0')
epoch 36 seql 38
testloss tensor(1.5284, device='cuda:0')
epoch 36 seql 38
testloss tensor(1.6498, device='cuda:0')
epoch 36 seql 38
testloss tensor(1.6002, device='cuda:0')
epoch 37 seql 38
tensor(1.4096, device='cuda:0', grad_fn=<MeanBackward0>)
epoch 37 seql 38
tensor(1.6279, device='cuda:0', grad_fn=<MeanBackward0>)
epoch 37 seql 38
tensor(1.6312, device='cuda:0', grad_fn=<MeanBackward0>)
epoch 37 seql 38
tensor(1.6436, device='cuda:0', grad_fn=<MeanBackward0>)
epoch 37 seql 38
tensor(1.7998, device='cuda:0', grad_fn=<MeanBackward0>)

In [ ]:
writer.flush()
writer.close()


In [ ]:
#%load_ext tensorboard

In [ ]:
#!kill 3285
#%tensorboard --logdir runs

In [ ]:
#!nvidia-smi

In [ ]:
with torch.no_grad():
  #model = model.eval()
  DonutDataset.displayCanvas('lstmpredictions.png',pred_dataset, model = model)

In [ ]:
for name, param in model.named_parameters():
  print(name)

In [ ]:
with torch.no_grad():
      #model = model.eval()
      for x,y in loader_test:
        h0 = None
        c0 = None
        for i in range(seql):
          x = x.cuda()
          y = y.cuda()
          
          if i == 0:
            out, h0, c0,o,output = model(x,y[:,0,:2],h0,c0,ind = i)
          else:
            out, h0, c0,o,output = model(x,y[:,0,:2],h0,c0,ind = i,o = o,output=output)
          
          loss = torch.mean((out-y[:,1+(25*i):1+(25*(i+1)),:2])**2)
        
          assert not torch.any(torch.isnan(out)).item()
          assert not torch.any(torch.isnan(y)).item()
        
        print('epoch',epoch,'seql',seql)
        print('testloss',loss)
  
        